# Case 1: Оптовые поставки зернового кофе

Компания, занимающаяся оптовыми поставками зернового кофе, запросила отчёт о продажах по месяцам. Интересуют только те продажи, у которых не было доставки:
* их количество в месяц (Count),
* процент плохих отзывов (Weight),
* доходность таких транзакций – в разрезе на продажи с вводом клиентом промокода и без (Promo).

Имеется выгрузка всех продаж с момента основания компании.

Необходимо написать код на Python, который прочтет выгрузку, проведет необходимые расчеты и запишет таблицу-результат в эксель.

*Поля итоговой таблицы:*
**«Месяц» – «Введен промокод» – «Кол-во транзакций» – «Плохих отзывов, %» - «Доходность, %».**

Примечание:
Доходность можно вычислить по формуле:

**Доходность, % = (Оплаченная по транзакциям сумма - Закупочная цена поставленных зерен) / Закупочная цена поставленных зерен * 100**,
где *Закупочная цена поставленных зерен = сумма Cost всех с нужными характеристиками*,
*Оплаченная по транзакциям сумма= сумма Paid транзакций с нужными характеристиками*.


In [314]:
import os

import pandas as pd

## 1) Загрузка данных

In [315]:
directory = r"C:\\Users\\HOME\\PycharmProjects\\case_studies_da\\1. Coffee_Beans_Supply"

In [316]:
data_path = "C:\\Users\\HOME\\PycharmProjects\\case_studies_da\\1. Coffee_Beans_Supply\\data_coffee_beans_supply.csv"

In [317]:
d = pd.read_csv("C:\\Users\\HOME\\PycharmProjects\\case_studies_da\\1. Coffee_Beans_Supply\\data_coffee_beans_supply.csv", encoding="iso-8859-1")
print(d)

      TransactionID\tDate\tCount\tCost\tÄîñòàâêà\tÏëîõîé îòçûâ\tPaid\tWeight\tPromo\tAutoIssue  \
0           1\t2020-Aug-21 8:53:03\t67\t6325\t0\t0\t2605                                         
1           2\t2019-Sep-3 11:53:03\t90\t8307\t1\t1\t8553                                         
2           3\t2019-Oct-15 4:31:02\t59\t9819\t0\t1\t9995                                         
3          4\t2019-Aug-26 4:14:00\t14\t5351\t1\t1\t12603                                         
4           5\t2020-Aug-27 1:21:03\t14\t7779\t1\t0\t5625                                         
...                                                  ...                                         
14360   14361\t2019-Oct-30 4:42:01\t70\t4716\t1\t1\t3201                                         
14361  14362\t2019-Oct-8 23:23:03\t14\t4547\t1\t0\t10810                                         
14362  14363\t2020-Aug-17 18:25:00\t8\t9940\t1\t1\t10597                                         
14363   14364\t2020-

In [318]:
rows = []
with open(data_path, 'r', encoding='Latin-1') as f:
    # Skip the first line, which contains the column headers
    f.readline()
    for line in f:
        # Split the line on tabs
        values = line.strip().split('\t')
        # Append the values to the list of rows
        rows.append(values)

df = pd.DataFrame(rows, columns=['TransactionID', 'Date', 'Count', 'Cost', 'Äîñòàâêà', 'Ïëîõîé îòçûâ', 'Paid', 'Weight', 'Promo', 'AutoIssue'])
df.head(10)

,TransactionID,Date,Count,Cost,Äîñòàâêà,Ïëîõîé îòçûâ,Paid,Weight,Promo,AutoIssue
0,"""1",2020-Aug-21 8:53:03,67,6325,0,0,"2605"",""0",5481,0,"0"""
1,"""2",2019-Sep-3 11:53:03,90,8307,1,1,"8553"",""0",5956,1,"1"""
2,"""3",2019-Oct-15 4:31:02,59,9819,0,1,"9995"",""0",5170,1,"0"""
3,"""4",2019-Aug-26 4:14:00,14,5351,1,1,"12603"",""0",8826,0,"0"""
4,"""5",2020-Aug-27 1:21:03,14,7779,1,0,"5625"",""0",5357,0,"0"""
5,"""6",2020-Aug-19 16:41:01,51,6595,1,1,"9191"",""0",8717,0,"0"""
6,"""7",2019-Aug-26 18:49:04,9,6764,1,1,"13167"",""0",9407,1,"1"""
7,"""8",2020-Aug-23 13:34:03,45,6381,0,0,"6251"",""0",9227,0,"0"""
8,"""9",2020-Aug-31 16:57:02,31,8795,0,1,"2047"",""0",6044,0,"1"""
9,"""10",2020-Aug-29 9:18:01,41,4175,0,1,"6300"",""0",7866,1,"1"""


## 2) Очистка данных

In [319]:
# Clean up TransactionID column
df['TransactionID'] = df['TransactionID'].str.replace('"', '')
df['AutoIssue'] = df['AutoIssue'].str.replace('"', '')

# Clean up Paid column
df['Paid'] = df['Paid'].str.replace('"', '').str.split(',', expand=True)[0]
df.columns

Index(['TransactionID', 'Date', 'Count', 'Cost', 'Äîñòàâêà', 'Ïëîõîé îòçûâ',
       'Paid', 'Weight', 'Promo', 'AutoIssue'],
      dtype='object')

In [320]:
df2 = df.drop(columns=['Äîñòàâêà', 'Ïëîõîé îòçûâ'])
df2

,TransactionID,Date,Count,Cost,Paid,Weight,Promo,AutoIssue
0,1,2020-Aug-21 8:53:03,67,6325,2605,5481,0,0
1,2,2019-Sep-3 11:53:03,90,8307,8553,5956,1,1
2,3,2019-Oct-15 4:31:02,59,9819,9995,5170,1,0
3,4,2019-Aug-26 4:14:00,14,5351,12603,8826,0,0
4,5,2020-Aug-27 1:21:03,14,7779,5625,5357,0,0
...,...,...,...,...,...,...,...,...
14360,14361,2019-Oct-30 4:42:01,70,4716,3201,7509,1,1
14361,14362,2019-Oct-8 23:23:03,14,4547,10810,8511,1,0
14362,14363,2020-Aug-17 18:25:00,8,9940,10597,9649,0,1
14363,14364,2020-Aug-27 9:38:01,92,7180,7200,6638,0,0


In [321]:
df2[['Count', 'Cost', 'Paid', 'Weight', 'Promo', 'AutoIssue']] = df2[['Count', 'Cost', 'Paid', 'Weight', 'Promo', 'AutoIssue']].astype(int)
df2

,TransactionID,Date,Count,Cost,Paid,Weight,Promo,AutoIssue
0,1,2020-Aug-21 8:53:03,67,6325,2605,5481,0,0
1,2,2019-Sep-3 11:53:03,90,8307,8553,5956,1,1
2,3,2019-Oct-15 4:31:02,59,9819,9995,5170,1,0
3,4,2019-Aug-26 4:14:00,14,5351,12603,8826,0,0
4,5,2020-Aug-27 1:21:03,14,7779,5625,5357,0,0
...,...,...,...,...,...,...,...,...
14360,14361,2019-Oct-30 4:42:01,70,4716,3201,7509,1,1
14361,14362,2019-Oct-8 23:23:03,14,4547,10810,8511,1,0
14362,14363,2020-Aug-17 18:25:00,8,9940,10597,9649,0,1
14363,14364,2020-Aug-27 9:38:01,92,7180,7200,6638,0,0


In [322]:
df2['Weight'] = df2['Weight'] / 100
df2

,TransactionID,Date,Count,Cost,Paid,Weight,Promo,AutoIssue
0,1,2020-Aug-21 8:53:03,67,6325,2605,54.81,0,0
1,2,2019-Sep-3 11:53:03,90,8307,8553,59.56,1,1
2,3,2019-Oct-15 4:31:02,59,9819,9995,51.70,1,0
3,4,2019-Aug-26 4:14:00,14,5351,12603,88.26,0,0
4,5,2020-Aug-27 1:21:03,14,7779,5625,53.57,0,0
...,...,...,...,...,...,...,...,...
14360,14361,2019-Oct-30 4:42:01,70,4716,3201,75.09,1,1
14361,14362,2019-Oct-8 23:23:03,14,4547,10810,85.11,1,0
14362,14363,2020-Aug-17 18:25:00,8,9940,10597,96.49,0,1
14363,14364,2020-Aug-27 9:38:01,92,7180,7200,66.38,0,0


In [323]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14365 entries, 0 to 14364
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TransactionID  14365 non-null  object 
 1   Date           14365 non-null  object 
 2   Count          14365 non-null  int32  
 3   Cost           14365 non-null  int32  
 4   Paid           14365 non-null  int32  
 5   Weight         14365 non-null  float64
 6   Promo          14365 non-null  int32  
 7   AutoIssue      14365 non-null  int32  
dtypes: float64(1), int32(5), object(2)
memory usage: 617.4+ KB


In [324]:
pd.DataFrame(round(df2.isna().mean()*100,)).style.background_gradient('coolwarm')

,0
TransactionID,0.000000
Date,0.000000
Count,0.000000
Cost,0.000000
Paid,0.000000
Weight,0.000000
Promo,0.000000
AutoIssue,0.000000


## 3) Создание новых признаков

* доходность по формуле;
* обработка даты из выгрузки (выделение месяца, года).

In [325]:
data = df2.copy()
data

,TransactionID,Date,Count,Cost,Paid,Weight,Promo,AutoIssue
0,1,2020-Aug-21 8:53:03,67,6325,2605,54.81,0,0
1,2,2019-Sep-3 11:53:03,90,8307,8553,59.56,1,1
2,3,2019-Oct-15 4:31:02,59,9819,9995,51.70,1,0
3,4,2019-Aug-26 4:14:00,14,5351,12603,88.26,0,0
4,5,2020-Aug-27 1:21:03,14,7779,5625,53.57,0,0
...,...,...,...,...,...,...,...,...
14360,14361,2019-Oct-30 4:42:01,70,4716,3201,75.09,1,1
14361,14362,2019-Oct-8 23:23:03,14,4547,10810,85.11,1,0
14362,14363,2020-Aug-17 18:25:00,8,9940,10597,96.49,0,1
14363,14364,2020-Aug-27 9:38:01,92,7180,7200,66.38,0,0


#### Рассчитаем доходность.

In [326]:
data['Profit'] = (data['Paid'] - data['Cost']) / data['Cost']
data

,TransactionID,Date,Count,Cost,Paid,Weight,Promo,AutoIssue,Profit
0,1,2020-Aug-21 8:53:03,67,6325,2605,54.81,0,0,-0.588142
1,2,2019-Sep-3 11:53:03,90,8307,8553,59.56,1,1,0.029614
2,3,2019-Oct-15 4:31:02,59,9819,9995,51.70,1,0,0.017924
3,4,2019-Aug-26 4:14:00,14,5351,12603,88.26,0,0,1.355261
4,5,2020-Aug-27 1:21:03,14,7779,5625,53.57,0,0,-0.276899
...,...,...,...,...,...,...,...,...,...
14360,14361,2019-Oct-30 4:42:01,70,4716,3201,75.09,1,1,-0.321247
14361,14362,2019-Oct-8 23:23:03,14,4547,10810,85.11,1,0,1.377392
14362,14363,2020-Aug-17 18:25:00,8,9940,10597,96.49,0,1,0.066097
14363,14364,2020-Aug-27 9:38:01,92,7180,7200,66.38,0,0,0.002786


#### Обработаем дату и время.

In [327]:
data['DateOnly'] = pd.to_datetime(data['Date']).dt.date
data

,TransactionID,Date,Count,Cost,Paid,Weight,Promo,AutoIssue,Profit,DateOnly
0,1,2020-Aug-21 8:53:03,67,6325,2605,54.81,0,0,-0.588142,2020-08-21
1,2,2019-Sep-3 11:53:03,90,8307,8553,59.56,1,1,0.029614,2019-09-03
2,3,2019-Oct-15 4:31:02,59,9819,9995,51.70,1,0,0.017924,2019-10-15
3,4,2019-Aug-26 4:14:00,14,5351,12603,88.26,0,0,1.355261,2019-08-26
4,5,2020-Aug-27 1:21:03,14,7779,5625,53.57,0,0,-0.276899,2020-08-27
...,...,...,...,...,...,...,...,...,...,...
14360,14361,2019-Oct-30 4:42:01,70,4716,3201,75.09,1,1,-0.321247,2019-10-30
14361,14362,2019-Oct-8 23:23:03,14,4547,10810,85.11,1,0,1.377392,2019-10-08
14362,14363,2020-Aug-17 18:25:00,8,9940,10597,96.49,0,1,0.066097,2020-08-17
14363,14364,2020-Aug-27 9:38:01,92,7180,7200,66.38,0,0,0.002786,2020-08-27


In [328]:
print(data['DateOnly'].isnull().sum())

0


In [329]:
data['DateOnly'] = pd.to_datetime(data['DateOnly'], format='%Y-%m-%d')

In [305]:
data[['Count', 'Cost', 'Paid', 'Weight', 'Promo', 'AutoIssue']] = data[['Count', 'Cost', 'Paid', 'Weight', 'Promo', 'AutoIssue']].astype(int)

In [306]:
# data.sort_index(inplace=True)
# data['DateOnly'].plot()
# plt.show();

In [307]:
def make_features(data, column, max_lag, rolling_mean_size):
    # creating new features
    data['Year'] = data.index.year
    data['Month'] = data.index.month
    data['Day'] = data.index.day

    return data

In [308]:
data = data.set_index('DateOnly')
data_features = make_features(data, 'Profit', 1, 1)
# NAs are < 10%, so we can safely delete them
data_features = data_features.dropna()
data_features.shape

(14365, 12)

In [309]:
data.head(10)

,TransactionID,Date,Count,Cost,Paid,Weight,Promo,AutoIssue,Profit,Year,Month,Day
DateOnly,,,,,,,,,,,,
2020-08-21,1,2020-Aug-21 8:53:03,67,6325,2605,54,0,0,-0.588142,2020,8,21
2019-09-03,2,2019-Sep-3 11:53:03,90,8307,8553,59,1,1,0.029614,2019,9,3
2019-10-15,3,2019-Oct-15 4:31:02,59,9819,9995,51,1,0,0.017924,2019,10,15
2019-08-26,4,2019-Aug-26 4:14:00,14,5351,12603,88,0,0,1.355261,2019,8,26
2020-08-27,5,2020-Aug-27 1:21:03,14,7779,5625,53,0,0,-0.276899,2020,8,27
2020-08-19,6,2020-Aug-19 16:41:01,51,6595,9191,87,0,0,0.393632,2020,8,19
2019-08-26,7,2019-Aug-26 18:49:04,9,6764,13167,94,1,1,0.946629,2019,8,26
2020-08-23,8,2020-Aug-23 13:34:03,45,6381,6251,92,0,0,-0.020373,2020,8,23
2020-08-31,9,2020-Aug-31 16:57:02,31,8795,2047,60,0,1,-0.767254,2020,8,31


#### Поля итоговой таблицы:
* «Месяц»
* «Введен промокод»
* «Кол-во транзакций»
* «Плохих отзывов, %»
* «Доходность, %»

In [310]:
data.columns

Index(['TransactionID', 'Date', 'Count', 'Cost', 'Paid', 'Weight', 'Promo',
       'AutoIssue', 'Profit', 'Year', 'Month', 'Day'],
      dtype='object')

In [311]:
data_final = data[['Month', 'Promo', 'Count', 'Weight', 'Profit']]
data_final

,Month,Promo,Count,Weight,Profit
DateOnly,,,,,
2020-08-21,8,0,67,54,-0.588142
2019-09-03,9,1,90,59,0.029614
2019-10-15,10,1,59,51,0.017924
2019-08-26,8,0,14,88,1.355261
2020-08-27,8,0,14,53,-0.276899
...,...,...,...,...,...
2019-10-30,10,1,70,75,-0.321247
2019-10-08,10,1,14,85,1.377392
2020-08-17,8,0,8,96,0.066097


#### Сохраним итоговые данные.

In [312]:
# create the directory if it does not exist
if not os.path.exists(directory):
    os.makedirs(directory)

# set the file path
file_path = os.path.join(directory, "data_final.csv")

# save the data as CSV file
data_final.to_csv(file_path, index=False)

### *Выводы:* Исходные данные были загружены, приведены к формату датафрейма и обработаны. В ходе предварительной обработки колонка ```'Weight'``` была приведена к процентному значению; были добавлены новые признаки - доходность```'Profit'``` была рассчитана по формуле, а также из колонки формата даты и времени были выделены значения года, месяца и дня ```'Year', 'Month', 'Day'```.

### Дальнейший анализ:
* выделить новые признаки с помощью ресемплинга;
* добавить скользящие среднее;
* проверить ряд на автокоррерляцию, оценить тренд, проследить сезонность.